##  Web Scraping Wiki

#### This notebook provides codes for webscrapping Wiki

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
#WebScraping
# Here, we're just importing both Beautiful Soup and the Requests library
page_link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
# this is the url that we've already determined is safe and legal to scrape from.
page_response = requests.get(page_link, timeout=5).text
# here, we fetch the content from the url, using the requests library
page_content = BeautifulSoup(page_response,'lxml' )

In [3]:
#Make DataFrame
data = []
table_body=page_content.find('tbody')
rows = table_body.find_all('tr')
for row in rows:
    cols=row.find_all('td')
    cols=[x.text.strip() for x in cols]
    data.append(cols)

In [4]:
#Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
col_name = ['PostalCode', 'Borough', 'Neighborhood']
df = pd.DataFrame(data, columns = col_name)
df = df[df['Borough'] != 'Not assigned']


In [5]:
#Check
df.head()

,PostalCode,Borough,Neighborhood
0,None,None,None
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park


In [6]:
#Drop blank header
df = df.drop(0, axis=0)

#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

In [7]:
dfgroupby = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()
dfgroupby.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [8]:
dfgroupby[dfgroupby['Neighborhood'] == 'Not assigned']

,PostalCode,Borough,Neighborhood
85,M7A,Queen's Park,Not assigned


In [9]:
dfgroupby.loc[85,'Neighborhood'] = "Queen's Park"

#### Only Select Borough contains Toronto

In [10]:
dfgroupby[dfgroupby['Borough'].str.contains('Toronto')]

,PostalCode,Borough,Neighborhood
37,M4E,East Toronto,The Beaches
41,M4K,East Toronto,"The Danforth West, Riverdale"
42,M4L,East Toronto,"The Beaches West, India Bazaar"
43,M4M,East Toronto,Studio District
44,M4N,Central Toronto,Lawrence Park
45,M4P,Central Toronto,Davisville North
46,M4R,Central Toronto,North Toronto West
47,M4S,Central Toronto,Davisville
48,M4T,Central Toronto,"Moore Park, Summerhill East"
49,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi..."


#### Print the number of rows of dataframe.

In [11]:
dfgroupby.shape

(103, 3)